In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

# Add Postag to Dataset

## Example

In [6]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [7]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del validation_dataset
del test_dataset

# Extract Feature

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Feature Extraction

In [9]:
X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_validation_dataset
del postagged_test_dataset

# Load Word Embedding and Add to The Feature

In [10]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0].lower()
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word.lower()']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [11]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model.txt')
X_cbow = deepcopy(X_test)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

del cbow_embedding_dict

In [12]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model.txt')
X_sg = deepcopy(X_test)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

del sg_embedding_dict

In [13]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model.txt')
X_fasttext = deepcopy(X_test)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

del fasttext_embedding_dict

In [14]:
del X_test

In [15]:
X_cbow[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.33895117,
 'w2': -0.014671952,
 'w3': 0.12068345,
 'w4': 1.0225383,
 'w5': -0.053509504,
 'w6': 0.34612843,
 'w7': -0.08877626,
 'w8': -0.060150824,
 'w9': 1.2847248,
 'w10': -0.11388345,
 'w11': -0.6135935,
 'w12': -0.49278504,
 'w13': 0.39436784,
 'w14': -0.073285446,
 'w15': 0.11112971,
 'w16': -0.22777088,
 'w17': 0.16231729,
 'w18': 0.4461143,
 'w19': 0.0011776277,
 'w20': -0.11573486,
 'w21': 0.08180463,
 'w22': -0.5032938,
 'w23': 0.73418355,
 'w24': -0.14306991,
 'w25': 0.90126806,
 'w26': -0.18855056,
 'w27': -0.63122505,
 'w28': 0.0043563913,
 'w29': -0.14975902,
 'w30': -0.97802

In [16]:
X_sg[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.053311832,
 'w2': 0.13628335,
 'w3': -0.36908904,
 'w4': 0.6710917,
 'w5': -0.043364376,
 'w6': -0.2286375,
 'w7': -0.04323148,
 'w8': 0.08788449,
 'w9': 0.8637854,
 'w10': -0.38204917,
 'w11': -0.26855293,
 'w12': -0.4236501,
 'w13': 0.5600748,
 'w14': 0.09522313,
 'w15': 0.43043205,
 'w16': -0.48882216,
 'w17': 0.064819306,
 'w18': 0.07685631,
 'w19': 0.052390438,
 'w20': -0.6501731,
 'w21': 0.15192252,
 'w22': -0.2691555,
 'w23': 0.5165376,
 'w24': -0.058252573,
 'w25': 0.8312122,
 'w26': -0.0033740667,
 'w27': 0.19275776,
 'w28': 0.039960757,
 'w29': 0.21826,
 'w30': -0.51947653,
 'w31

In [17]:
X_fasttext[0][0]

{'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': -0.53241414,
 'w2': -0.6670444,
 'w3': 0.19736609,
 'w4': -0.23998305,
 'w5': -0.7144021,
 'w6': 0.53846425,
 'w7': -0.27698493,
 'w8': -0.041880284,
 'w9': -0.24590936,
 'w10': -0.23515925,
 'w11': -0.24975732,
 'w12': 0.36505768,
 'w13': -0.8391249,
 'w14': -0.22660933,
 'w15': 0.7303037,
 'w16': 0.29804635,
 'w17': -0.041736826,
 'w18': 0.26564932,
 'w19': -0.26180515,
 'w20': 0.13123806,
 'w21': -0.0027916746,
 'w22': -0.21876915,
 'w23': 0.25730494,
 'w24': 0.015905615,
 'w25': 0.13872473,
 'w26': 0.4867511,
 'w27': 0.16061094,
 'w28': 0.30005667,
 'w29': 0.28330767,
 'w30': 0.16052526,

# Load Model

In [18]:
import pickle

In [19]:
filename_cbow = 'temp_crf_cbow.sav'
loaded_model_cbow = pickle.load(open(filename_cbow, 'rb'))

filename_sg = 'temp_crf_sg.sav'
loaded_model_sg = pickle.load(open(filename_sg, 'rb'))

filename_fasttext = 'temp_crf_fasttext.sav'
loaded_model_fasttext = pickle.load(open(filename_fasttext, 'rb'))

In [20]:
labels = list(loaded_model_cbow.classes_)
labels.remove('O')
labels

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

# CBOW

In [21]:
y_pred_cbow = loaded_model_cbow.predict(X_cbow)
metrics.flat_f1_score(y_test, y_pred_cbow,
                      average='weighted', labels=labels)

0.8113356671266855

# SG

In [22]:
y_pred_sg = loaded_model_sg.predict(X_sg)
metrics.flat_f1_score(y_test, y_pred_sg,
                      average='weighted', labels=labels)

0.8136969886497679

# FastText

In [23]:
y_pred_fasttext = loaded_model_fasttext.predict(X_fasttext)
metrics.flat_f1_score(y_test, y_pred_fasttext,
                      average='weighted', labels=labels)

0.8155770941766326